In [3]:
import pandas as pd
import openpyxl
from sqlalchemy import create_engine, text
from datetime import datetime, timedelta
import pyodbc
import urllib.parse

### Functions

In [18]:
def load_excel(filename):
    wb = openpyxl.load_workbook(filename, read_only=True)
    ws = wb['Sheet1']
    header_row_idx = None
    for i, row in enumerate(ws.iter_rows(max_col=2, max_row=10, values_only=True)):
        if row and 'Case Number' in row:
            header_row_idx = i
            break
    wb.close()
    if header_row_idx is not None:
        df = pd.read_excel(filename, sheet_name='Sheet1', skiprows=header_row_idx)
        return df
    else:
        raise ValueError(f"Header row with 'Case Number' not found in: {filename}")
    
def convert_to_date(df):
    dtimeFields = ['Case Date', 'Case Submission Date','Latest Action Date','Transferred to Geospatial','GEO Completion','GEO S Completion','Transferred to Ops', 'Attachment Added Date', "ListDate"]
    for field in dtimeFields:
        if field in df.columns:
            df[field] = pd.to_datetime(df[field]).dt.date
    return df

### DB Configurations

In [23]:
# Define config at the top of the file
AppDB_CONFIG = {
    "server": '0003-MAAL-01\\LASSQLSERVER',
    "database": 'LASCaseWorkerApp',
    "username": 'LASCaseWorker',
    "password": 'LASCaseWorker'
}

# Utility function to create a connection
def get_connection_Sql():
    return pyodbc.connect(
        f"DRIVER={{SQL Server}};"
        f"SERVER={AppDB_CONFIG['server']};"
        f"DATABASE={AppDB_CONFIG['database']};"
        f"UID={AppDB_CONFIG['username']};"
        f"PWD={AppDB_CONFIG['password']};"
    )

## Dashboard DB SQL
DashDB_CONFIG = {
    "server": '0003-MAAL-01\\LASSQLSERVER',
    "database": 'GRSDASHBOARD',
    "username": 'lasapp',
    "password": 'lasapp@LAS123'
}

# Build ODBC connection string from existing DB_CONFIG
odbc_params = (
    "DRIVER={ODBC Driver 17 for SQL Server};"
    f"SERVER={DashDB_CONFIG['server']};"
    f"DATABASE={DashDB_CONFIG['database']};"
    f"UID={DashDB_CONFIG['username']};"
    f"PWD={DashDB_CONFIG['password']};"
)

DashPost = {
    "server":"10.150.40.74",
    "port": '5432',
    "database": "GSA",
    "username": "app_user",
    "password": "app1234"
}
## Dashboard DB PostgreSQL
connection_str_post = f"postgresql://{DashPost['username']}:{DashPost['password']}@{DashPost['server']}:{DashPost['port']}/{DashPost['database']}"

In [24]:
odbc_connect_str = urllib.parse.quote_plus(odbc_params)
# Create SQLAlchemy engine for SQL Server via pyodbc
engine_sqlserver = create_engine(f"mssql+pyodbc:///?odbc_connect={odbc_connect_str}", fast_executemany=True)
engine_postgres = create_engine(connection_str_post)
tables = ['ApprovedCases', 'CR_Current', 'CR_Data', 'ClassificationData', 'CurrentCases', 'EditorsList', 'GeoData', 'GeoSCompletionData', 'HistoricalData', 'MG_Current', 'MG_Data', 'OpsData', 'RejectedCancelled', 'ReturnedCases', 'SR_Current', 'SR_Data', 'ST_EditorList', 'Ticketing', 'TransferToGeoData', 'Urgent', 'VIP',]

In [ ]:
query_sql = """SELECT * FROM grsdbrd."{}" """
query_post = """SELECT * FROM public."{}" """

def join_userlist(comp_df, editorlist):
    comp_df['GEO S Completion'] = pd.to_datetime(comp_df['GEO S Completion']).dt.normalize()
    editorlist = editorlist.rename({'CaseProtalName': 'Geo Supervisor'},axis=1)
    editorlist["ListDate"] = pd.to_datetime(editorlist["ListDate"]).dt.normalize()
    comp_df = comp_df.sort_values(by=["GEO S Completion", "Geo Supervisor"])
    editorlist = editorlist.sort_values(by=["ListDate", "Geo Supervisor"])
    comp_df = pd.merge_asof(comp_df, editorlist, by="Geo Supervisor", left_on="GEO S Completion", 
                            right_on="ListDate", direction='backward')
    comp_df['GEO S Completion'] = [pd.to_datetime(i).date() for i in comp_df['GEO S Completion']]
    comp_df['ListDate'] = [pd.to_datetime(i).date() for i in comp_df['ListDate']]
    return comp_df


def generate_evaluation_sheet(engine, start_date, end_date):
    query = query_post+ """WHERE "GEO S Completion" BETWEEN '{}' AND '{}' """
    value = query.format(tables[7], str(start_date), str(end_date))
    completed = convert_to_date(pd.read_sql(value, engine))
    editorList = convert_to_date(pd.read_sql(query_post.format(tables[5]), engine_postgres))
    completed = join_userlist(completed, editorList)
    completed = completed.dropna('Geo Supervisor Recommendation')
    return completed[completed['Geo Supervisor Recommendation'].str.contains('يعاد')]
end = datetime.now().date()
start = end - timedelta(days=7)
compCases = generate_evaluation_sheet(engine_postgres,start, end)


ValueError: Cannot mask with non-boolean array containing NA / NaN values

In [42]:
compCases.columns

Index(['Case Number', 'Absolute Ownership', 'Duplicate Case',
       'Generated Titles', 'Case Submission Date', 'Latest Action Date',
       'Action', 'Assignee', 'Transferred to Geospatial', 'Return To Geo Team',
       'Count of Returns Cases ', 'GEO Completion', 'GEO S Completion',
       'Transferred to Ops', 'Case Status', 'REN', 'Boundary Length Deed',
       'Boundary Length Parcel', 'MoJ Deed Number', 'Moj Real Estate Serial',
       'MoJ Plan #', 'MoJ Real Estate Area', 'MoJ Land Number', 'City Name',
       'District Name', 'MoJ Deed Area Text', 'Property Type',
       'Parcel Area Size', 'Parcel Number (PCP)', 'Location Description',
       'Attachments', 'Attachment Added Date', 'Deed East Limit Description',
       'Deed East Limit Length', 'Deed Eastern Type',
       'Deed  West Limit Description', 'Deed West Limit Length',
       'Deed Western Boarder Type', 'Deed North Limit Description',
       'Deed North Limit Length', 'Deed Northern Boarder Type',
       'Deed Sout